In [12]:
########################################################################################################
#                                                                                                      #
#                                 VERSIÓN 2 OCR TICKETS WALMART                                        #
#                                                                                                      #
#                                      ISMAEL MARCOS MUÑOZ                                             #
#                                          12-04-2019                                                  #
#                                                                                                      #
########################################################################################################


try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract
import re
import pandas as pd
import numpy as np
from datetime import datetime
pytesseract.pytesseract.tesseract_cmd = r'/usr/local/bin/tesseract'


class ImageParser:

    def __init__(self, PathImage):
        self.PathImage = PathImage
        self.ReadImage()
        
    def main(self):
        d = dict()
        d['Date'] = self.ParseDate()
        d['ItemsSold'] = self.ParseItemsSold()
        d['Manager'] = self.ParseManager()
        d['Total'] = self.ParseTotal()
        d['Account'] = self.ParseAccount()
        return d
        

    def ReadImage(self):
        self.OCRText = pytesseract.image_to_string(Image.open(self.PathImage))

    def Show(self):
        self.Show = print(pytesseract.image_to_string(Image.open(self.PathImage)))

    def ParseDate(self):
        self.Fecha = re.findall('\d{1,2}\/\d{1,2}\/\d{1,2}\s\d{1,2}:\d{1,2}:\d{1,2}', self.OCRText)
        
        #self.Fecha = self.Fecha[0] + [None]
        
        if len(self.Fecha)==0:
            self.Fecha = None
        else:
            self.Fecha = self.Fecha[0]
            self.Fecha = datetime.strptime(self.Fecha, '%m/%d/%y %H:%M:%S') #PARA PASAR EL STRING A DATETIME
        
        
        
        # self.Fecha = re.findall('(\d{1,2}\/\d{1,2}\/\d{1,2})\s(\d{1,2}:\d{1,2}:\d{1,2})', self.OCRText)
        
        # return "La fecha de compra es {o.Fecha}".format(o=self)
        return self.Fecha

    def ParseItemsSold(self):
        self.ItemsSold = re.findall('ITE\wS\sSOLD\s([0-9]+)', self.OCRText)
        
        
        if len(self.ItemsSold)==0:
            self.ItemsSold = None
        else:
            self.ItemsSold = self.ItemsSold[0]
            self.ItemsSold = int(self.ItemsSold)


        return self.ItemsSold

    def ParseManager(self):
        self.Manager = re.findall('\wANAGER\s(\w{0,}\s\w{0,})', self.OCRText)
        
        self.Manager = ''.join(self.Manager)  #Pasamos la lista a string por ser un nombre


        return self.Manager

    def ParseTotal(self):
        self.Total = re.findall('TOTAL\s([0-9]+\D[0-9]+)', self.OCRText)

        self.Total= (self.Total + [None])[0]
        
#         if len(self.Total)==0:
#             self.Total = None
#         else:
#             self.Total = self.Total[0]
#             self.Total = float(self.Total)

        return self.Total

    def ParseAccount(self):
        self.Account = re.findall('ACCOUNT\s([0-9]+)', self.OCRText)
        
        if len(self.Account)==0:
            self.Account = None
        else:
            self.Account = self.Account[0]
            self.Account = int(self.Account)


        return self.Account

In [13]:
paths = [r'/Users/imm1711/Desktop/DATA SCIENTIST/OCR V2/walmart_1.jpg',r'/Users/imm1711/Desktop/DATA SCIENTIST/OCR V2/walmart_2.jpg',
         r'/Users/imm1711/Desktop/DATA SCIENTIST/OCR V2/walmart_3.jpg']

#d = dict()
df = pd.DataFrame(index = np.linspace(1,len(paths),len(paths)) , columns=['Date','ItemsSold','Manager','Total','Account'])

for i in range(len(paths)):
    
        instancia = ImageParser(paths[i])
        instancia.main()
    
        df.iloc[i,0] = instancia.ParseDate()       
        df.iloc[i,1] = instancia.ParseItemsSold()
        df.iloc[i,2] = instancia.ParseManager()        
        df.iloc[i,3] = instancia.ParseTotal()
        df.iloc[i,4] = instancia.ParseAccount()
        instancia = ImageParser(paths[i])
        

df


,Date,ItemsSold,Manager,Total,Account
1.0,2010-10-14 21:20:50,6,IRENE BROWN,None,605214515600
2.0,2010-10-14 21:20:50,6,IRENE BROWN,10.30,605214515600
3.0,None,2,,None,None
